## Import Necessary Libraries

In [1]:
import GPy
import numpy as np
import pandas as pd
import time
# Set Seed
np.random.seed(0)

## Read Train And Test Files

#### Make Sure you keep all these files in the current directory

In [2]:
# Read Files
X_train = np.load('X_train.npy') 
X_test  = np.load('X_test.npy')
Y_train = np.load('Y_train.npy')
Y_test  = np.load('Y_test.npy') 

# Make Y_train and Y_test as 2d array for model function fitting
Y_train = Y_train[:,None]
Y_test = Y_test[:,None]

## Model Building and Training

In [3]:
start_time = time.time() # To save start time of execution of code

#Name For saving model
model_name = 'GPR_Model'

try:
    
    # Intialize Dimensions
    Dimensions = X_train.shape[1]

    # Kernel Building 
    k_rbf = GPy.kern.RBF(input_dim=Dimensions, variance=1, lengthscale=None, ARD=True
                           , active_dims=None, name='rbf', useGPU=False, inv_l=False) 

    # Model Building
    model_rbf = GPy.models.GPRegression(X_train,Y_train,k_rbf)  

    # Optmization of Model
    model_rbf.optimize(optimizer=None,start=None,messages=False, max_iters=1000
                              , ipython_notebook=True, clear_after_finish=True) 
    
    # Save the model for later use
    model_rbf._save_model(model_name, compress=False) 
    # Set compress = False to save model as .json file otherwise it will be saved as .zip file

except:
    print('Jitter Error')
    # If jitter error occurs try to reset np.random.seed() with some other value
    
end_time = time.time()    
print("Execution Time is ", end_time-start_time, "seconds")

Execution Time is  2.0544962882995605 seconds


## Prediction Module: Load pre-trained model to get prediction on test set

In [4]:
# Set alpha factor
alpha = 1  ## Here alpha = 1 , is used, you can use it depending upon the choice and problem 

In [5]:
# DataFrame for Prediction Result
result = pd.DataFrame(columns=['Predicted_Score','Rank'])

In [6]:
### Get the Pre-trained model json file
model_name= 'GPR_Model.json'

# load Pre-trained Model 
model_rbf = GPy.core.model.Model.load_model(model_name)

#  Get Predictions
pred_mean,pred_var= model_rbf.predict(X_test, full_cov=False, Y_metadata=None, kern=None,
                                      likelihood=None, include_likelihood=False) 

# Save Prediction results
prediction_mean = pred_mean.flatten()
prediction_var =  pred_var.flatten()
# Set negative varience occured due to numerical error change it to Zero(0)
prediction_var[prediction_var<0]=0
prediction_std =  np.sqrt(prediction_var)
## Save sum of Mean and alpha*Std in one variable , this is our Score for Ranking
Prediction_sum = prediction_mean + alpha* prediction_std


# Save result in dataframe
result['Predicted_Score'] = Prediction_sum

### Generate Ranking  by Sorting  Score in Descending

In [7]:
result = result.sort_values('Predicted_Score',ascending=False)
result['Rank'] =  range(1,len(result)+1)
result.reset_index(drop=True,inplace=True)
result.head()

,Predicted_Score,Rank
0,1.459370,1
1,1.262126,2
2,1.195904,3
3,1.165598,4
4,1.156953,5


In [8]:
# Save Result as .txt file
result.to_csv('Ranking_result.txt',sep='\t',index=False)